<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLGymGA-CarttPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI]((https://gym.openai.com)

Voir la page d'[introduction à Gym](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLIntroGym.ipynb).

### Installation de gym

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gym

In [1]:
import gym
import numpy as np
from random import randint

#### <font color=red>Sous colab</font>
*Sous colab, l'affichage ne sera pas dynamique.. Il faudra passer soit par un rendu sous forme de plot (matplotlib), soit passer par une vidéo à enregistrer...*

***Solution pour affichage statique***<br>
*Il vous faut importer, seulement si vous utilisez colab, les packages suivants :*

In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install x11-utils

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

---
## Charger l' environnement
On utilise ici l'environnement `CartPole-v1`.

In [58]:
env = gym.make('CartPole-v1') 

---
### Les valeurs des caractéristiques
Pour `CartPole-v1`, vous trouverez l'explication des valeurs [ici](https://www.gymlibrary.ml/pages/environments/classic_control/cart_pole).

**Actions**

| Valeur | Action | 
|:--:|:--:|
| 0 | aller à gauche | 
| 1 | aller à droite | 


**Observations**

| No | Observation | Min | Max |
|:--:|:--:|:--:|:--:|
| 0 | Position | -4.8 | 4.8 |
| 1 | Velocité | -inf | +inf |
| 2 | Angle du mat (radian) | -0.418 | +0.418 |
| 3 | Velocité anglulaire du mat | -inf | +inf |

Vérification : 

In [59]:
print(env.spec)
print("nombre et type d'actions = ", env.action_space) 
print("nombre et type d'observations = ", env.observation_space) 
print("valeurs de  observations les plus basses = ", env.observation_space.low) 
print("valeurs de observations les plus elevées = ", env.observation_space.high) 

EnvSpec(CartPole-v1)
nombre et type d'actions =  Discrete(2)
nombre et type d'observations =  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
valeurs de  observations les plus basses =  [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
valeurs de observations les plus elevées =  [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


### (Re)Initialiser l'environnement
`env.reset()` initialise les variables d'environnement. L'exécution est obligatoire après avoir chargé l'environnement. Ici ce sont la position de départ et la vitesse.
Pour avoir un environnement non déterministe, par défaut `env.reset()` affecte des valeurs entre  -0.05 et +0.05 pour chaque champs observable : 

In [60]:
env.reset()

array([ 0.02057121,  0.02878238, -0.03504172,  0.02839395], dtype=float32)

Mais ici nous souhaitons partir du même point (0) à vitesse nulle, mais avec un angle en radian du mat de 0.1 rad (mat penché légèrement à droite) à chaque test d'une séquence d'actions. Avant chaque évaluation de séquence d'actions, ***on n'utilisera pas*** `env.reset()` par la suite et on écrira donc : 

In [61]:
env.env.state= np.array([0., 0., 0.1, 0.])
env.render()

True

---
### Les actions
- Dans `CartPole-v1` : l'action 0 déplace le véhicule sur la gauche, 1 le déplace sur la droite.

- ***Exécuter une action :***
  - *exécution* : `observation, reward,done,info = env.step(action)`
    - `observation`: ici la position et vitesse du 'Wagon' et du mat ([Wx, Wv, Ax, Mv])
    - `reward`: utilité de l'état (récompense)
    - `done` : booléen (True si but atteint)
    - `info` : éventuelle information retournée par l'outil

In [75]:
#test de 10 déplacements à droite
env.env.state= np.array([0., 0., 0.1, 0.])
action = 1
for _ in range(0, 10):
    env.render()
    observation, reward, done, info = env.step(action)        
    print("Wx={:2.2}m, \tWv={:2.2}, \tMangle={:2.2}° rad, \tMv={:2.2}".format(observation[0], observation[1], observation[2], observation[3]))
print("________________________________")


Wx=0.0m, 	Wv=0.19, 	Mangle=0.1° rad, 	Mv=-0.26
Wx=0.0039m, 	Wv=0.39, 	Mangle=0.095° rad, 	Mv=-0.52
Wx=0.012m, 	Wv=0.58, 	Mangle=0.084° rad, 	Mv=-0.78
Wx=0.023m, 	Wv=0.77, 	Mangle=0.069° rad, 	Mv=-1.0
Wx=0.039m, 	Wv=0.97, 	Mangle=0.048° rad, 	Mv=-1.3
Wx=0.058m, 	Wv=1.2, 	Mangle=0.022° rad, 	Mv=-1.6
Wx=0.081m, 	Wv=1.4, 	Mangle=-0.01° rad, 	Mv=-1.9
Wx=0.11m, 	Wv=1.6, 	Mangle=-0.048° rad, 	Mv=-2.2
Wx=0.14m, 	Wv=1.7, 	Mangle=-0.091° rad, 	Mv=-2.5
Wx=0.17m, 	Wv=1.9, 	Mangle=-0.14° rad, 	Mv=-2.8
________________________________


In [76]:
#test de 10 déplacements à gauche
env.env.state= np.array([0., 0., 0, 0.])
action = 0
for _ in range(0, 10):
    env.render()
    observation, reward, done, info = env.step(action)
    print("Wx={:2.2}m, \tWv={:2.2}, \tMangle={:2.2}° rad, \tMv={:2.2}".format(observation[0], observation[1], observation[2], observation[3]))
print("________________________________")


Wx=0.0m, 	Wv=-0.2, 	Mangle=0.0° rad, 	Mv=0.29
Wx=-0.0039m, 	Wv=-0.39, 	Mangle=0.0059° rad, 	Mv=0.59
Wx=-0.012m, 	Wv=-0.59, 	Mangle=0.018° rad, 	Mv=0.88
Wx=-0.023m, 	Wv=-0.78, 	Mangle=0.035° rad, 	Mv=1.2
Wx=-0.039m, 	Wv=-0.98, 	Mangle=0.059° rad, 	Mv=1.5
Wx=-0.059m, 	Wv=-1.2, 	Mangle=0.088° rad, 	Mv=1.8
Wx=-0.082m, 	Wv=-1.4, 	Mangle=0.12° rad, 	Mv=2.1
Wx=-0.11m, 	Wv=-1.6, 	Mangle=0.17° rad, 	Mv=2.4
Wx=-0.14m, 	Wv=-1.8, 	Mangle=0.22° rad, 	Mv=2.8
Wx=-0.18m, 	Wv=-2.0, 	Mangle=0.27° rad, 	Mv=3.1
________________________________


---
### La récompense (reward) et le but
Le *reward* dépend l'environnement d'exécution.. Dans `CartPole-v1`, un point est accordé par action, avec un seuil à 475

Le *but* dans `CartPole-v0` est : 
  - que le mat reste le plus longtemps dans un angle de +-12° radian (0 étant la position verticale)
  - que le wagon reste dans le cadre (position entre -4.8m et 4.8m)
  - d'atteindre 500 actions (200 si l'environnement est `CartPole-v0`)


---

# Test d'algorithme génétique
## Test de programme évolutionnaire pour atteindre l'objectif
- Utiliser donc l'environnement `CartPole-v1`
- Programmez (en python) un algo génétique avec initialement les éléments suivants : 
  - sequence : tableau de valeurs entières 0 ou 1.
  - 1 point de croisement (au centre)
    - *(pour commencer, on pourrait proposer 2 points de croisement (au 1/3, et 2/3))*
  - taille sequence : taille_seq = 50 initialement, puis, 100, 200, 500
  - taille population initiale : taille_pop = 500, que l'on peut réduire ou augmenter
  - taux de gènes mutants dans une séquence : taux_mut_seq = 0.1
  - taux de séquences mutantes dans la population : taux_mut_pop = 0.2
  - nombre max de cycles de reproductions : nb_cycles = 3000


- Pour tester une séquence, il suffit de balayer le tableau et de lancer l'action correspondante..(en la transformant en un tableau de réel (1 -> [1.0])
- Les 4 meilleures séquences (11,22,33,44) se reproduisent entre elles, donnant donc 12 fils : 12, 13, 14, 21, 23, 24, 31, 32, 34, 41, 42, 43.
- Idéalement, un filtre doit permettre de supprimer les séquences identiques qui pourraient venir des croisements et des mutations.

- Tester votre algo sur le nb de générations et afficher le rendu toutes les 100 générations et bien sûr le rendu de la meilleure séquence finale.
  - (uniquement la position finale *colab*)
  - (sous forme d'animation sous autre outil)


- ***NB:*** 
  - Au plus simple il est possible d'utiliser une population (liste) de couples [sequence, utilite] où l'utilité serait la distance en x.
  
  

In [89]:
#exemple :  
env.reset()
env.env.state= np.array([0, 0, 0.1, 0])
tab = np.random.randint(0, 2, 10) 
couple = [tab, 0]
print(couple)
recompense = 0
for a in couple[0]:
    obs, reward, done, info = env.step(a)
    if not done: recompense += reward
print("arrive en x=", round(obs[0],2), ", fini=", done)
couple[1] = recompense
print("couple [actions, arrivee] = ", couple)

[array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1]), 0]
arrive en x= -0.06 , fini= True
couple [actions, arrivee] =  [array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1]), 9.0]


---
### Guide : 
- Créer votre fonction de récompense pour favoriser les séquences qui maintiennent le mat le plus longtemps dans la norme demandée (+-12°)
- Basez vous sur l'exemple complet utilisant l'environnement Moutain-Car pour développer l'algorithme et fournir les courbes d'évolution